In [1]:
# aid.ipynb
# Author: Kyle Larson
# Purpose top secret
import numpy as np 
import soundfile as sf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
# Stack overflow was consulted in order to parse '.flac' 
#path = '../input/rfcx-species-audio-detection/train/00204008d.flac'
#data, samplerate = sf.read(path) #this method is thanks to Harry Moreno 
#from his Stack Overflow response on June 13 '18 at the following URL:
#https://stackoverflow.com/questions/50804170/load-flac-file-in-python-same-as-scipy-or-librosa


In [2]:
import os
import pandas as pd
file_name_list = os.listdir('C:/Users/admin/anaconda3/01 PROJECTS/Audio ID')
#the following sequential open strategy was found not to work due to frequency mismatch between id_data and filenames
ignore = ['.git', '.ipynb_checkpoints', 'train.zip']
audio_data = pd.DataFrame({file_name_list[2]:[]})
for i in range(len(file_name_list)):#len(id_data) #10 files ~ 5 seconds w/ RAM ~25%
    if(file_name_list[i]!=ignore[0] and file_name_list[i]!=ignore[1] and file_name_list[i]!=ignore[2]):
        data, samplerate = sf.read('C:/Users/admin/anaconda3/01 PROJECTS/Audio ID/' +file_name_list[i])
        audio_data[file_name_list[i]] = np.array(data)
    if(i==10):
        break
audio_data.head() 

,00204008d.flac,003b04435.flac,003bec244.flac,005f1f9a5.flac,006ab765f.flac,0072f0839.flac,0079ff47b.flac,007f87ba2.flac,00834f88e.flac
0,0.007538,0.002106,0.008301,0.015076,-0.009308,-0.001404,-0.028168,0.047791,-0.067291
1,0.007111,0.007935,0.028137,0.026093,-0.009949,-0.000092,0.016174,0.030731,-0.066895
2,0.006165,0.005188,0.024689,0.029877,-0.011444,0.003693,0.070160,0.016846,-0.059387
3,0.010773,0.006744,0.019836,0.038025,-0.010559,-0.004700,0.078705,-0.004059,-0.030823
4,0.004669,0.004028,-0.002319,0.042023,-0.011383,0.002777,0.102417,-0.026215,0.000427


In [3]:
#at 1 min 5 sec the following error occured:
#Your notebook tried to allocate more memory than is available. It has restarted.
#thats 65 seconds or 130 files that can occupy ram at once

In [4]:
len(audio_data.loc[0])

9

In [5]:
#the plan is to use a fft (Fast Forier Transform) to halve the number of datapoints to reduce compute time
#then use an SVD (singular value decomposition) to prioritize the points that are principle components
from scipy.fft import fft, ifft
fft_aid =[]
for i in range(2, len(audio_data.loc[0])):#2 is to avoid auto '.git' undesirables, depends on intividual system
    x = np.array(audio_data.loc[:][file_name_list[i]])
    fft_aid += [fft(x)]
print(fft_aid)
    

#time for this 9 was <1sec

[array([41.13421631-0.j        , 10.64642932+0.25820614j,
        9.2332184 -0.19021899j, ...,  7.33564754-1.87112699j,
        9.2332184 +0.19021899j, 10.64642932-0.25820614j]), array([40.94290161 -0.j        , -7.09001743+15.60606578j,
       -3.8866546  +6.41422956j, ...,  0.11746973 -7.08458867j,
       -3.8866546  -6.41422956j, -7.09001743-15.60606578j]), array([73.42666626 -0.j        , -7.19791793+10.73032501j,
        6.07912089+16.33527279j, ..., 10.57079271 -7.26532391j,
        6.07912089-16.33527279j, -7.19791793-10.73032501j]), array([ 338.96380615 -0.j        , -195.86090338+22.05340036j,
        234.4222708 -23.29121672j, ..., -221.97790813-58.74850902j,
        234.4222708 +23.29121672j, -195.86090338-22.05340036j]), array([12.26766968-0.j        , -0.78639117-5.08030292j,
        5.61737918+2.27025603j, ..., 10.45377886+5.74960348j,
        5.61737918-2.27025603j, -0.78639117+5.08030292j]), array([-23.9730835 -0.j        ,  14.18005352-0.28015113j,
        16.20927771-

In [6]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
half_fft_aid = []
num_ids = 23
def drop_second_half(array):
    array = array[0:int(np.ceil((len(array)/2)))]
    return array

for i in range(len(fft_aid)):
    #print(fft_aid[i])
    #print(len(fft_aid[i][:])) -> 2880000 and 1440000 alternating
    half_fft_aid += [drop_second_half(fft_aid[i][:])]
    inshape = len(fft_aid[i][:])
    model = keras.Sequential([
        #hidden
        #layers.Dense(units=inshape+2, activation='relu', input_shape=[inshape]),
        #ResourceExhaustedError: OOM when allocating tensor with shape[2880000,2880002] 
        #and type float on /job:localhost/replica:0/task:0/device:CPU:0 
        #by allocator cpu [Op:RandomUniform]
        #hidden
    ])
    
#hidden = []
#for i in range(inshape-num_ids):
#    hidden += layers.Dense(units=inshape+2-i, activation='relu')
#'Dense' object is not iterable


In [7]:
print(str(max(half_fft_aid[0][:])))
print(' i:')
print(np.argmax(half_fft_aid[0][:]))
print(np.argmax(half_fft_aid[0][:])/len(half_fft_aid[0]))
print("one twelfth is")
print(1/12)
print("in points that is")
print(round(len(half_fft_aid[0])/12))

(515.2424333451936+187.7523291372073j)
 i:
1125000
0.78125
one twelfth is
0.08333333333333333
in points that is
120000


In [20]:
#pull the 120000 points needed straddle the max noise 
#(how this straddling can be done should be an experiment for improving the model)
#based on a few vocal lessons from the local university, 
#it is noted that the ressonance is not necessarily found at the beginning of the tone and so 
#2 seconds of straddle (2/5 of 120000 or 48000) before the max sample (ie max volume) 
#and 3 after will be the initial rule
index_of_max=0
lower_straddle_index_of_max=0
upper_straddle_index_of_max=0
straddled_half_fft_aid=[]
two_fifths = 48000
five_seconds = 120000
for i in range(len(half_fft_aid)):
    # find the max
    index_of_max = np.argmax(half_fft_aid[i])
    if(index_of_max - two_fifths > 0):
        lower_straddle_index_of_max = index_of_max - two_fifths
    else:
        lower_straddle_index_of_max = index_of_max
    upper_straddle_index_of_max = lower_straddle_index_of_max + five_seconds 
    #print("half fft aid"+' '+str(i)+' '+str(half_fft_aid[i]))
    #print("straddled"+str(half_fft_aid[i][lower_straddle_index_of_max:upper_straddle_index_of_max]))
    straddled_half_fft_aid.append(half_fft_aid[i][lower_straddle_index_of_max:upper_straddle_index_of_max])
    
print( straddled_half_fft_aid )
    

[array([-1.23641224-0.06714433j, -4.47748665+0.17023295j,
        0.61693184-2.32679992j, ..., -8.06654671-1.21958179j,
        1.03759671+0.08221637j,  3.03556187-4.37742067j]), array([-2.16540592+5.37554847j, -1.60660318-1.83198365j,
       -4.1466158 -1.58630858j, ...,  1.13490689-2.89797798j,
       -6.15460119+1.73488673j, -0.93077876+1.53264254j]), array([-3.44092032+6.82566274j,  2.03769369-4.68230678j,
        0.89893959+7.68022338j, ..., -6.30457096+3.94157568j,
        1.87612199-2.64354849j,  0.35708174-0.49438135j]), array([ 1115.7874276 +635.24892048j, -1078.32741271-793.7990298j ,
         993.37084709+829.43936471j, ...,  -142.87868768 -57.31507743j,
          50.91234292-182.7883046j ,    66.95363977+174.88064702j]), array([811.02943534-2.96983414e+01j, 186.40701054+8.35320329e+00j,
       188.57945027-6.29475699e+02j, ...,  -2.83682952+5.45709856e+00j,
        -3.08219613-1.59974759e-01j,   8.02820523-5.64908096e-01j]), array([ -8.85394419 +3.11121342j, -20.03734029 -1

In [16]:
max(half_fft_aid[3])
np.argmax(half_fft_aid[3])

3685

In [9]:
len(half_fft_aid)

7

In [21]:
from numpy import array
from scipy.linalg import svd
from numpy import dot
from numpy import diag
from numpy import zeros

print(straddled_half_fft_aid)
U, s, VT = svd(straddled_half_fft_aid)
Sigma = zeros(straddled_half_fft_aid.shape[0], straddled_half_fft_aid.shape[1])
Sigma[:straddled_half_fft_aid.shape[1], :straddled_half_fft_aid.shape[1]] = diag(s)
B = U.dot(Sigma.dot(VT))
print(B)

[array([-1.23641224-0.06714433j, -4.47748665+0.17023295j,
        0.61693184-2.32679992j, ..., -8.06654671-1.21958179j,
        1.03759671+0.08221637j,  3.03556187-4.37742067j]), array([-2.16540592+5.37554847j, -1.60660318-1.83198365j,
       -4.1466158 -1.58630858j, ...,  1.13490689-2.89797798j,
       -6.15460119+1.73488673j, -0.93077876+1.53264254j]), array([-3.44092032+6.82566274j,  2.03769369-4.68230678j,
        0.89893959+7.68022338j, ..., -6.30457096+3.94157568j,
        1.87612199-2.64354849j,  0.35708174-0.49438135j]), array([ 1115.7874276 +635.24892048j, -1078.32741271-793.7990298j ,
         993.37084709+829.43936471j, ...,  -142.87868768 -57.31507743j,
          50.91234292-182.7883046j ,    66.95363977+174.88064702j]), array([811.02943534-2.96983414e+01j, 186.40701054+8.35320329e+00j,
       188.57945027-6.29475699e+02j, ...,  -2.83682952+5.45709856e+00j,
        -3.08219613-1.59974759e-01j,   8.02820523-5.64908096e-01j]), array([ -8.85394419 +3.11121342j, -20.03734029 -1

MemoryError: Unable to allocate 215. GiB for an array with shape (120000, 120000) and data type complex128

In [ ]:
import numpy as np
def drop_second_half(array):
    array = array[0:int(np.ceil((len(array)/2)))]
    return array

test=[1,2,3,4,5,6,7]

print(drop_second_half(test))